In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/InfoVisProject/confusionflow

/content/gdrive/MyDrive/InfoVisProject/confusionflow


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import VGG16

In [4]:
from callback import Logs

# Load Data

In [5]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# Model 1

In [6]:
model_1 = Sequential(
    [
     Flatten(),
     Dense(256, activation='relu'),
     Dropout(0.2),
     Dense(128, activation='relu'),
     Dropout(0.2),
     Dense(10, activation='softmax'),
    ]
)

In [7]:
model_1_callback = Logs((x_test, y_test), loss='sparse_categorical_crossentropy')

In [8]:
model_1.compile(
    optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [9]:
model_1.fit(x_train, y_train, epochs=35, batch_size=64, callbacks=[model_1_callback])

Epoch 1/35
782/782 [==============================] - 3s 3ms/step - loss: 2.0198 - accuracy: 0.2645
Epoch 2/35
782/782 [==============================] - 2s 3ms/step - loss: 1.8560 - accuracy: 0.3360
Epoch 3/35
782/782 [==============================] - 2s 3ms/step - loss: 1.7851 - accuracy: 0.3633
Epoch 4/35
782/782 [==============================] - 2s 3ms/step - loss: 1.7385 - accuracy: 0.3785
Epoch 5/35
782/782 [==============================] - 2s 3ms/step - loss: 1.7002 - accuracy: 0.3968
Epoch 6/35
782/782 [==============================] - 2s 3ms/step - loss: 1.6694 - accuracy: 0.4078
Epoch 7/35
782/782 [==============================] - 2s 3ms/step - loss: 1.6413 - accuracy: 0.4199
Epoch 8/35
782/782 [==============================] - 2s 3ms/step - loss: 1.6190 - accuracy: 0.4259
Epoch 9/35
782/782 [==============================] - 2s 3ms/step - loss: 1.5979 - accuracy: 0.4343
Epoch 10/35
782/782 [==============================] - 2s 3ms/step - loss: 1.5780 - accuracy: 0.4410

In [10]:
model_1_callback.export('../logs', 'model_1')

# Model 2

In [11]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg.trainable = False

In [12]:
model_2 = Sequential()
model_2.add(vgg)
model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dense(10, activation='softmax'))

model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
model_2.compile(
    optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

In [14]:
model_2_callback = Logs((x_test, y_test), loss='sparse_categorical_crossentropy')

In [15]:
model_2.fit(x_train, y_train, epochs=35, batch_size=64, callbacks=[model_2_callback])

Epoch 1/35
782/782 [==============================] - 11s 13ms/step - loss: 1.3618 - accuracy: 0.5229
Epoch 2/35
782/782 [==============================] - 10s 12ms/step - loss: 1.1715 - accuracy: 0.5900
Epoch 3/35
782/782 [==============================] - 10s 13ms/step - loss: 1.0969 - accuracy: 0.6165
Epoch 4/35
782/782 [==============================] - 10s 13ms/step - loss: 1.0452 - accuracy: 0.6352
Epoch 5/35
782/782 [==============================] - 10s 13ms/step - loss: 0.9999 - accuracy: 0.6499
Epoch 6/35
782/782 [==============================] - 10s 13ms/step - loss: 0.9619 - accuracy: 0.6634
Epoch 7/35
782/782 [==============================] - 10s 13ms/step - loss: 0.9231 - accuracy: 0.6789
Epoch 8/35
782/782 [==============================] - 10s 13ms/step - loss: 0.8865 - accuracy: 0.6900
Epoch 9/35
782/782 [==============================] - 10s 13ms/step - loss: 0.8546 - accuracy: 0.7004
Epoch 10/35
782/782 [==============================] - 10s 13ms/step - loss: 0.821

In [16]:
model_2_callback.export('../logs', 'model_2')